In [11]:
SEASON = '2009-10'

In [12]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,LeBron James,CLE,SF,22.441491,25,76,76,39.0,10.1,20.1,...,33.5,13.3,5.2,18.5,0.299,9.1,2.8,11.8,10.3,61.0
2,Kevin Durant,OKC,SF,16.154669,21,82,82,39.5,9.7,20.3,...,32.0,11.1,5.0,16.1,0.238,6.3,0.8,7.1,7.5,50.0
3,Dwyane Wade,MIA,SG,15.623964,28,77,77,36.3,9.3,19.6,...,34.9,8.5,4.6,13.0,0.224,7.1,2.1,9.2,7.9,47.0
4,Dwight Howard,ORL,C,11.869927,24,82,82,34.7,6.2,10.2,...,23.9,6.1,7.1,13.2,0.223,1.6,2.8,4.4,4.6,59.0
5,Dirk Nowitzki,DAL,PF,11.301325,31,81,80,37.5,8.9,18.5,...,28.8,8.4,3.9,12.3,0.194,3.7,0.5,4.2,4.7,55.0
6,Tim Duncan,SAS,C,10.704796,33,78,77,31.3,7.2,13.9,...,26.0,6.4,4.5,10.9,0.215,4.4,1.8,6.1,5.0,50.0
7,Pau Gasol,LAL,C,10.620129,29,65,65,37.0,7.0,13.0,...,21.4,6.8,4.2,11.0,0.220,3.7,1.3,4.9,4.2,57.0
8,Kobe Bryant,LAL,SG,10.583730,31,73,73,38.8,9.8,21.5,...,32.3,5.4,4.0,9.4,0.160,3.9,0.2,4.1,4.4,57.0
9,Josh Smith,ATL,PF,9.908809,24,81,81,35.4,6.2,12.3,...,22.2,4.2,5.1,9.3,0.155,2.4,2.7,5.1,5.1,53.0
10,Deron Williams,UTA,PG,9.474818,25,76,76,36.9,6.5,13.9,...,23.8,7.4,3.0,10.3,0.177,4.2,-0.4,3.8,4.1,53.0
